# Tutorial 2.4: Three-Body Potential Energy Surface

Next, we will parameterize the "physical baseline" ($V_{phys}$) of the PEF.

In this section, we construct, from scratch, a 1-body MB-nrg-style PEF for CH3-NH2.

## 2.1.0 Definitions and Imports

In [1]:
%run Tutorial_2_Imports.ipynb

In [10]:
%run Tutorial_2_Paths.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Let's generate the second set of configurations for our training set, those from metadynamics configurations.

## II.1.? Parallel-Bias Metadynamics Configuration Generation

In [4]:
definition_CH3NH2 = read_definition(paths.definitions.systems.CH3NH2)
minimized_structure_CH3NH2 = read_system(definition_CH3NH2, paths.structures.minimized_CH3NH2)

definition_H2O = read_definition(paths.definitions.systems.H2O)
minimized_structure_H2O = read_system(definition_H2O, paths.structures.minimized_H2O)

definition_CH3NH2_H2O_H2O = read_definition(paths.definitions.systems.CH3NH2_H2O_H2O)

trimer_struture = read_systems(definition_CH3NH2_H2O_H2O, paths.clusters.CH3NH2_H2O_H2O)[0]

In [5]:
bond_params, angle_params, nonbonded_params = get_pbmetad_parameters(
        trimer_struture
)

In [6]:
bond_params

{('A', 'B'): (30, 1.090410024347401),
 ('A', 'C'): (50, 1.4843142427207179),
 ('C', 'D'): (30, 1.0355461195163942),
 ('E', 'F'): (30, 0.9895526574811143)}

In [7]:
angle_params

{('B', 'A', 'C'): (10, 110.73761574767536),
 ('D', 'C', 'D'): (10, 105.76182010902501),
 ('B', 'A', 'B'): (10, 108.16415660139323),
 ('A', 'C', 'D'): (10, 108.21981201112177),
 ('F', 'E', 'F'): (10, 101.40817132883848)}

In [8]:
nonbonded_params

{'A': (0.03, 2.4),
 'B': (0.01, 1.2),
 'C': (0.03, 2.4),
 'D': (0.01, 1.2),
 'E': (0.03, 2.4),
 'F': (0.01, 1.2)}

In [9]:
charges = read_parameters(paths.Vphys.charges)
charges = {symmetry: charges[symmetry]*1/3 for symmetry in charges}
charges["E"] = -0.6*1/3
charges["F"] = +0.3*1/3

In [ ]:
perform_pbmetad_simulation(
        lammps_executable_path,
        [minimized_structure_CH3NH2, minimized_structure_H2O, minimized_structure_H2O],
        num_configs=500000,
        sample_interval=10,
        temperature=700,
        seed=12345,
        configurations_path=paths.PIP_CH3NH2_H2O_H2O.configs.pbmetad_pool,
        bond_params=bond_params,
        angle_params=angle_params,
        nonbonded_params=nonbonded_params,
        charges=charges,
        pbmetad_workdir=paths.scratch.pbmetad_CH3NH2_H2O_H2O,
        restart_path=paths.restart.pbmetad_CH3NH2_H2O_H2O
)